In [0]:
#imports:
import pandas as pd
import numpy as np
import math
import os
import json 

import xgboost
from sklearn.metrics import r2_score

In [0]:
PREP_DATA_PATH = "./drive/My Drive/НИР/data/prepared_facebook_data.csv"
data=pd.read_csv(PREP_DATA_PATH)

RES_SAVE_DIR = "./drive/My Drive/НИР/data/results/test 3"

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
# get some data
X, y = data.iloc[:,:-1].values, data.iloc[:,-1].values

In [0]:
dtrain = xgboost.DMatrix(X, label=y)

In [0]:
# https://www.kaggle.com/nanomathias/bayesian-optimization-of-xgboost-lb-0-9769 
# http://krasserm.github.io/2018/03/21/bayesian-optimization/

In [0]:
!pip install scikit-optimize

     |████████████████████████████████| 81kB 2.2MB/s 


In [0]:
from skopt import BayesSearchCV
from skopt.space import Real, Space, Integer

bayes_cv_opt = BayesSearchCV(
    estimator = xgboost.XGBRegressor(
      tree_method = "gpu_hist", 
      gpu_id = 0, 
      verbosity = 0
    ),
    search_spaces = {
        'min_child_weight': Integer(1, 10),
        'max_depth': Integer(3, 10),
        'subsample': Real(0.5, 0.9, 'uniform'),
        'colsample_bytree': Real(0.5, 0.9, 'uniform'),
        'reg_lambda': Real(0.0, 1.0, 'uniform'),
        'reg_alpha': Real(0.0, 1.0, 'uniform'),        
    },    
    scoring = 'r2',
    cv = 5,
    n_iter = 729,
    iid=False, 
    optimizer_kwargs={
        "acq_func": "EI"
        }
)
#print(bayes_cv_opt.total_iterations)

# callback handler

def on_step(optim_result):
    score = bayes_cv_opt.best_score_
    print("Best score: %s" % score)
    print("Best iteration index ", bayes_cv_opt.best_index_)
    #if score >= 0.98:
    #    print('Interrupting!')
    #    return True

bayes_cv_opt.fit(X, y, callback=on_step)

print("val. score: %s" % bayes_cv_opt.best_score_)


In [0]:
# n_initial_points int, default=10 
#  skopt.space.Space(
# acq_func string, default=`”gp_hedge”` "EI" for negative expected improvement.
# https://scikit-optimize.github.io/stable/modules/generated/skopt.Optimizer.html#skopt.Optimizer

In [0]:
#gpyopt:
"""
#!pip install gpyopt
#from GPyOpt.methods import BayesianOptimization


def my_r2_score(preds, dtrain):
    labels = dtrain.get_label()
    return 'r2', r2_score(labels, preds)

def bayes_opt_tune_xgb(para):
  para = para[0]
  params = {
      "tree_method": "gpu_hist", 
      "gpu_id": 0, 
      "verbosity": 0,
      "max_depth": int(round(para[0])),
      "min_child_weight": int(round(para[1])),
      "alpha": para[2],
      "lambda": para[3],
      "subsample": para[4],
      "colsample_bytree": para[5]
      }
    #Cross validating with the specified parameters in 5 folds
  cv_result = xgboost.cv(params, dtrain, nfold=5, feval=my_r2_score,  maximize=True)
  gpyopt_output.append(
        [
         cv_result['test-r2-mean'].iloc[-1],
         params["max_depth"], params["min_child_weight"], 
         params["alpha"], params["lambda"], 
         params["subsample"], params["colsample_bytree"]
        ]
    )

  return cv_result['test-r2-mean'].iloc[-1]


gpyopt_output = []
gpy_importance = pd.DataFrame()
bds = [ {'name': 'max_depth', 'type': 'discrete', 'domain': (3, 10)},
        {'name': 'min_child_weight', 'type': 'discrete', 'domain': (1, 10)},
        {'name': 'alpha', 'type': 'continuous', 'domain': (0, 1)},
        {'name': 'lambda', 'type': 'continuous', 'domain': (0, 1)},
        {'name': 'subsample', 'type': 'continuous', 'domain': (0.5, 0.9)},
        {'name': 'colsample_bytree', 'type': 'continuous', 'domain': (0.5, 0.9)}
      ]
optimizer = BayesianOptimization(f=bayes_opt_tune_xgb, 
                                 domain=bds,
                                 model_type='GP',
                                 optimize_restarts = 72,
                                 initial_design_numdata = 10,
                                 acquisition_type ='EI',
                                 maximize=True)

optimizer.run_optimization(max_iter=719)
results = pd.DataFrame(gpyopt_output)
print(results.max(axis = 0))
"""

In [0]:
"""
# https://hyperopt.github.io/hyperopt/
!pip install hyperopt
import xgboost as xgb

from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials
from hyperopt import hp
import numpy as np
from sklearn.metrics import mean_squared_error


# XGB parameters
xgb_reg_params = {
    "tree_method": "gpu_hist", 
    "gpu_id": 0, 
    "verbosity": 0,
    'max_depth':        hp.choice('max_depth',        np.arange(3, 10, 1, dtype=int)),
    'min_child_weight': hp.choice('min_child_weight', np.arange(1, 10, 1, dtype=int)),
    'alpha': hp.uniform('alpha', 0.0, 1),
    'lambda': hp.uniform('lambda', 0.0, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 0.9),
    'subsample':        hp.uniform('subsample', 0.5, 0.9),
    
}

xgb_para = dict()
xgb_para['reg_params'] = xgb_reg_params

def my_r2_score(preds, dtrain):
    labels = dtrain.get_label()
    return 'r2', r2_score(labels, preds)

class HPOpt(object):

    def __init__(self, dtrain):
        self.dtrain = dtrain


    def process(self, fn_name, space, trials, algo, max_evals):
        fn = getattr(self, fn_name)
        try:
            result = fmin(fn=fn, space=space, algo=algo, max_evals=max_evals, trials=trials)
        except Exception as e:
            return {'status': STATUS_FAIL,
                    'exception': str(e)}
        return result, trials

    def bayes_opt_tune_xgb(self, para):
        cv_result = xgboost.cv(params = para['reg_params'], 
                               dtrain = self.dtrain, 
                               nfold=5, feval=my_r2_score,  maximize=True)
        loss = cv_result = 1 - cv_result['test-r2-mean'].iloc[-1]
        return {'loss': loss, 'status': STATUS_OK}

obj = HPOpt(dtrain)
trials=Trials()
# algo=tpe.suggest 0.35029
xgb_opt = obj.process(fn_name='bayes_opt_tune_xgb', space=xgb_para, trials=trials, algo=tpe.suggest, max_evals=729)

print(xgb_opt)
print (1-min(trials.losses()))
"""